# GloVe file process

This part is convert the GloVe format into Word2vec format, but I only find how to convert the .text file.

In [1]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.scripts.glove2word2vec import get_glove_info

In [2]:
glove_file = 'GloVe.txt'
get_glove_info(glove_file)

(57593, 150)

In [3]:
tmp_file = get_tmpfile("test_word2vec.txt")
_ = glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)

In [4]:
get_glove_info(tmp_file)

(57594, 1)

In [5]:
tmp_file

'/var/folders/dz/vk151wrn1kd7rc48nwv46hqw0000gn/T/test_word2vec.txt'

# word2vec 

This notebook is equivalent to `demo-word.sh`, `demo-analogy.sh`, `demo-phrases.sh` and `demo-classes.sh` from Google.

In [1]:
%load_ext autoreload
%autoreload 2

## Training

In [12]:
!sed -e "s/’/'/g" -e "s/′/'/g" -e "s/''/ /g" < evaluation.txt | tr -c "A-Za-z'_ \n" " " > evaluation-norm0.txt

In [13]:
word2vec.word2phrase('evaluation-norm0.txt', 'evaluation-norm0-phrase0.txt', threshold=1000000, verbose=True)

Starting training using file evaluation-norm0.txt

Vocab size (unigrams + bigrams): 25278
Words in train file: 72680


In [14]:
word2vec.word2phrase('evaluation-norm0-phrase0.txt', 'evaluation-norm0-phrase1.txt', threshold=50000000, verbose=True)

Starting training using file evaluation-norm0-phrase0.txt

Vocab size (unigrams + bigrams): 25278
Words in train file: 72680


In [16]:
!tr A-Z a-z < evaluation-norm0-phrase1.txt > evaluation-norm1-phrase1.txt

In [2]:
import word2vec

In [42]:
word2vec.word2vec

<function word2vec.scripts_interface.word2vec(train, output, size=100, window=5, sample='1e-3', hs=0, negative=5, threads=12, iter_=5, min_count=5, alpha=0.025, debug=2, binary=1, cbow=1, save_vocab=None, read_vocab=None, verbose=False)>

Delete punctuations, numbers

In [3]:
!sed -e "s/’/'/g" -e "s/′/'/g" -e "s/''/ /g" < Training | tr -c "A-Za-z'_ \n" " " > Training-norm0

In [4]:
#time ./word2phrase -train Training-norm0 -output Training-norm0-phrase0 -threshold 200 -debug 2

In [5]:
word2vec.word2phrase('Training-norm0', 'Training-norm0-phrase0', threshold=100, verbose=True)

Starting training using file Training-norm0
Words processed: 1500K     Vocab size: 265K  
Vocab size (unigrams + bigrams): 166640
Words in train file: 1516658


Run `word2phrase` to group up similar words "Los Angeles" to "Los_Angeles"

In [1]:
word2vec.word2phrase('Training-norm0-phrase0', 'Training-norm0-phrase1', threshold=50, verbose=True)
!tr A-Z a-z < Training-norm0-phrase1 > Training-norm1-phrase1

NameError: name 'word2vec' is not defined

In [7]:
#time ./word2vec -train Training-norm1-phrase1 -output vectors-phrase.bin -cbow 1 -size 200 -window 10 -negative 25 -hs 0 -sample 1e-5 -threads 20 -binary 1 -iter 15
#./distance vectors-phrase.bin


This created a `-phrases` file that we can use as a better input for `word2vec`.


In [92]:
word2vec.word2vec('Training-norm1-phrase1', 'vectors-phrase.bin', cbow=1, size=150, window=10,sample="1e-5", negative=25, threads=20, iter_=15, min_count=1, verbose=True)



Starting training using file Training-norm1-phrase1
Vocab size: 57593
Words in train file: 3850498
Alpha: 0.000002  Progress: 100.05%  Words/thread/sec: 118.82k  sec: 106.34k  hread/sec: 118.81k  75%  Words/thread/sec: 118.73k  ress: 93.02%  Words/thread/sec: 118.76k  679  Progress: 93.30%  Words/thread/sec: 118.72k  ha: 0.001609  Progress: 93.58%  Words/thread/sec: 118.78k  73k  sec: 118.80k  

That created a `.bin` file containing the word vectors in a binary format.

Now we generate the clusters of the vectors based on the trained model.

That created a `text8-clusters.txt` with the cluster for every word in the vocabulary

## Predictions

In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import word2vec

Import the `word2vec` binary file created above

In [94]:
model = word2vec.load('vectors-phrase.bin')

In [7]:
import gensim

def load_phrase_vectors(model_file):
    model = gensim.models.KeyedVectors.load_word2vec_format(model_file, binary=False)
    return model
model_file='test_word2vec.txt'
model = load_phrase_vectors(model_file)


We can take a look at the vocabulary as a numpy array

In [8]:
model.vocab

{'the': <gensim.models.keyedvectors.Vocab at 0x121759be0>,
 'of': <gensim.models.keyedvectors.Vocab at 0x121759cf8>,
 'and': <gensim.models.keyedvectors.Vocab at 0x121759d68>,
 'in': <gensim.models.keyedvectors.Vocab at 0x121759dd8>,
 'to': <gensim.models.keyedvectors.Vocab at 0x121759e48>,
 'a': <gensim.models.keyedvectors.Vocab at 0x121759c88>,
 'with': <gensim.models.keyedvectors.Vocab at 0x121759eb8>,
 'for': <gensim.models.keyedvectors.Vocab at 0x121759f28>,
 'was': <gensim.models.keyedvectors.Vocab at 0x121759f98>,
 'children': <gensim.models.keyedvectors.Vocab at 0x121759fd0>,
 'were': <gensim.models.keyedvectors.Vocab at 0x121767080>,
 'that': <gensim.models.keyedvectors.Vocab at 0x1217670f0>,
 'is': <gensim.models.keyedvectors.Vocab at 0x121767160>,
 'on': <gensim.models.keyedvectors.Vocab at 0x1217671d0>,
 'or': <gensim.models.keyedvectors.Vocab at 0x121767240>,
 'are': <gensim.models.keyedvectors.Vocab at 0x1217672b0>,
 'by': <gensim.models.keyedvectors.Vocab at 0x121767320>

Or take a look at the whole matrix

In [9]:
model.vectors.shape

(57593, 150)

In [10]:
model.vectors

array([[-0.670133,  0.292567,  0.820156, ...,  0.022716, -0.421597,
        -0.509952],
       [-0.695665, -0.402831, -0.683536, ...,  0.036169,  0.789463,
        -0.402949],
       [-0.959403, -0.137432, -0.447992, ..., -0.614785, -0.198532,
        -1.422503],
       ...,
       [ 0.058225,  0.196555,  0.055162, ...,  0.13145 , -0.084235,
         0.17543 ],
       [ 0.043517,  0.205058,  0.174117, ..., -0.071412,  0.020059,
         0.247611],
       [ 0.061655,  0.057846,  0.036607, ...,  0.014345, -0.023932,
         0.134173]], dtype=float32)

We can retreive the vector of individual words

In [12]:
model['autism'].shape

(150,)

In [13]:
model['asd'][:10]

array([ 0.259025, -0.410931, -1.020677,  0.636955,  0.486279,  0.034095,
        0.039554, -0.163851, -0.68576 ,  0.150924], dtype=float32)

We can calculate the distance between two or more (all combinations) words.

In [14]:
model.distance("asd", "autism")

0.40864866971969604

# The structure of phenotype calculation


In [17]:
x = "medical_history_asdpto" 
y = "social_competence_asdpto"

In [28]:
model.distance(x, b0), model.distance(b0, b00), model.distance(x, b00)

(0.7852630913257599, 0.7994900494813919, 0.7973895370960236)

In [18]:
dic = {}
b0 = "comorbidities_asdpto"
#b1 = "complaints_and_indications_asdpto" 
b1 = "diagnosis_asdpto"
b2 = "exposures_asdpto"
b3 = "perinatal_history_asdpto"
dic["b0"] = b0
dic["b1"] = b1
dic["b2"] = b2
dic["b3"] = b3

c0 = "interpersonal_interactions_asdpto"
dic["c0"] = c0

In [19]:
b00 = "autoimmune_diseases_asdpto"
b01 = "congenital_abnormalities_asdpto"
b02 = "mental_disorders_asdpto"
b03 = "ear_diseases_asdpto"
b04 = "cardiovascular_diseases_asdpto"
#c4 = "endocrine_system_diseases_asdpto"
b05 = "gastrointestinal_diseases_asdpto"
b06 = "mental_disorders_asdpto"
#c7 = "nervous_system_diseases_asdpto"
#c8 = "virus_diseases_asdpto"

b10 = "onset_of_symptoms_asdpto"
b11 = "primary_diagnosis_asdpto"

b20 = "dietary_restrictions_asdpto"
b21 = "hospitalizations_asdpto"
b22 = "lead_exposure_asdpto"
b23 = "medications_asdpto"
b24 = "vaccination_asdpto"

b30 = "apgar_score_asdpto"
b31 = "pregnancy_outcome_asdpto"

c00 = "social_anxiety_asdpto"
c01 = "social_awkwardness_asdpto"
c02 = "social_interest_asdpto"

dic["b00"] = b00
dic["b01"] = b01
dic["b02"] = b02
dic["b03"] = b03
dic["b04"] = b04
dic["b05"] = b05
dic["b06"] = b06
dic["b10"] = b10
dic["b11"] = b11
dic["b20"] = b20
dic["b21"] = b21
dic["b22"] = b22
dic["b23"] = b23
dic["b24"] = b24
dic["b30"] = b30
dic["b31"] = b31
dic["c00"] = c00
dic["c01"] = c01
dic["c02"] = c02


In [21]:
model[c0].shape

(150,)

In [22]:
print(model.distance(y, dic["b0"], dic["b00"]))

TypeError: distance() takes 3 positional arguments but 4 were given

In [155]:
t = 0
s = 0
for i in range(0, 4):
    m = "b" + str(i)
    if i == 0:
        z = 7
    if i == 1:
        z = 2
    if i == 2:
        z = 5
    if i == 3:
        z = 2
    for j in range(0,z):
        n = m + str(j)
        s += 1
        #print(m,n)
        print("\n")
        q = model.distance(x, dic[m], dic[n])[0][2]
        w = model.distance(x, dic[m], dic[n])[1][2]
        e = model.distance(x, dic[m], dic[n])[2][2]
        if q>w and e>w:
            print("True")
            t += 1
        else:
            print("False", model.distance(x, dic[m], dic[n]))

for i in range(0,3):
    n = "c0" + str(i)
    s += 1
    q = model.distance(y, dic["c0"], dic[n])[0][2]
    w = model.distance(y, dic["c0"], dic[n])[1][2]
    e = model.distance(y, dic["c0"], dic[n])[2][2]
    if q>w and e>w:
        print("True")
        print("\n")
        t += 1
    else:
        print("False", model.distance(x, dic["c0"], dic[n]))
        print("\n")


print(t, s, t/s)



False [('medical_history_asdpto', 'comorbidities_asdpto', 0.4310047618930925), ('medical_history_asdpto', 'autoimmune_diseases_asdpto', 0.7155802334774808), ('comorbidities_asdpto', 'autoimmune_diseases_asdpto', 0.7181790799516077)]


False [('medical_history_asdpto', 'comorbidities_asdpto', 0.4310047618930925), ('medical_history_asdpto', 'congenital_abnormalities_asdpto', 0.8469884472646656), ('comorbidities_asdpto', 'congenital_abnormalities_asdpto', 0.47220057329296705)]


False [('medical_history_asdpto', 'comorbidities_asdpto', 0.4310047618930925), ('medical_history_asdpto', 'mental_disorders_asdpto', 0.432600955859777), ('comorbidities_asdpto', 'mental_disorders_asdpto', 0.9047669879667907)]


False [('medical_history_asdpto', 'comorbidities_asdpto', 0.4310047618930925), ('medical_history_asdpto', 'ear_diseases_asdpto', 0.9092150646361182), ('comorbidities_asdpto', 'ear_diseases_asdpto', 0.4372319899726096)]


False [('medical_history_asdpto', 'comorbidities_asdpto', 0.43100476

In [30]:
t = 0
s = 0
for i in range(0, 4):
    m = "b" + str(i)
    if i == 0:
        z = 7
    if i == 1:
        z = 2
    if i == 2:
        z = 5
    if i == 3:
        z = 2
    for j in range(0,z):
        n = m + str(j)
        s += 1
        #print(m,n)
        print("\n")
        q = model.distance(x, dic[m])
        w = model.distance(x, dic[n])
        e = model.distance(dic[m], dic[n])
        if q>w and e>w:
            print("True")
            t += 1
        else:
            print("False")
            print(x, dic[m], q)
            print(x, dic[n], w)
            print(dic[m], dic[n], e)

for i in range(0,3):
    n = "c0" + str(i)
    s += 1
    q = model.distance(y, dic["c0"])
    w = model.distance(y, dic[n])
    e = model.distance(dic["c0"], dic[n])
    if q>w and e>w:
        print("True")
        print("\n")
        t += 1
    else:
        print("False")
        print(x, dic[m], q)
        print(x, dic[n], w)
        print(dic[m], dic[n], e)


print(t, s, t/s)



False
medical_history_asdpto comorbidities_asdpto 0.7852630913257599
medical_history_asdpto autoimmune_diseases_asdpto 0.7973895370960236
comorbidities_asdpto autoimmune_diseases_asdpto 0.7994900494813919


False
medical_history_asdpto comorbidities_asdpto 0.7852630913257599
medical_history_asdpto congenital_abnormalities_asdpto 0.898610033094883
comorbidities_asdpto congenital_abnormalities_asdpto 1.1107790991663933


False
medical_history_asdpto comorbidities_asdpto 0.7852630913257599
medical_history_asdpto mental_disorders_asdpto 0.8326137661933899
comorbidities_asdpto mental_disorders_asdpto 0.842248409986496


False
medical_history_asdpto comorbidities_asdpto 0.7852630913257599
medical_history_asdpto ear_diseases_asdpto 0.9275767654180527
comorbidities_asdpto ear_diseases_asdpto 1.2345630526542664


True


False
medical_history_asdpto comorbidities_asdpto 0.7852630913257599
medical_history_asdpto gastrointestinal_diseases_asdpto 1.0004838444292545
comorbidities_asdpto gastrointe

## Similarity



There is a helper function to create a combined response as a numpy [record array](http://docs.scipy.org/doc/numpy/user/basics.rec.html)

In [116]:
model.generate_response(indexes, metrics)

rec.array([('autism_hp', 0.97669083),
           ('autism_hp_spectrum_disorder', 0.93495965),
           ('a_systematic_review', 0.92398257),
           ('restricted_interests', 0.92049319),
           ('autism_hp_spectrum_disorders', 0.9196843 ),
           ('autism_spectrum_disorders_asds', 0.91861276),
           ('autism_spectrum_disorder_asd', 0.918133  ),
           ('autistic_spectrum_disorder', 0.91586191),
           ('autistic_behavior_hp_autistic_behavior_hp', 0.91522592),
           ('autism_hp_spectrum_disorder_atrial_septal_defect_hp', 0.91333033)],
          dtype=[('word', '<U78'), ('metric', '<f8')])